In [5]:
from fileinput import filename

import polars as pl
import polars.selectors as cs
import time
import chardet

from numpy.f2py.cfuncs import includes
from sqlalchemy.dialects.mssql.information_schema import columns

penguins = pl.read_csv("../data/penguins.csv")
print(penguins)

shape: (344, 9)
┌───────┬───────────┬───────────┬────────────────┬───┬───────────────┬─────────────┬────────┬──────┐
│ rowid ┆ species   ┆ island    ┆ bill_length_mm ┆ … ┆ flipper_lengt ┆ body_mass_g ┆ sex    ┆ year │
│ ---   ┆ ---       ┆ ---       ┆ ---            ┆   ┆ h_mm          ┆ ---         ┆ ---    ┆ ---  │
│ i64   ┆ str       ┆ str       ┆ str            ┆   ┆ ---           ┆ str         ┆ str    ┆ i64  │
│       ┆           ┆           ┆                ┆   ┆ str           ┆             ┆        ┆      │
╞═══════╪═══════════╪═══════════╪════════════════╪═══╪═══════════════╪═════════════╪════════╪══════╡
│ 1     ┆ Adelie    ┆ Torgersen ┆ 39.1           ┆ … ┆ 181           ┆ 3750        ┆ male   ┆ 2007 │
│ 2     ┆ Adelie    ┆ Torgersen ┆ 39.5           ┆ … ┆ 186           ┆ 3800        ┆ female ┆ 2007 │
│ 3     ┆ Adelie    ┆ Torgersen ┆ 40.3           ┆ … ┆ 195           ┆ 3250        ┆ female ┆ 2007 │
│ 4     ┆ Adelie    ┆ Torgersen ┆ NA             ┆ … ┆ NA            ┆ NA  

In [6]:
penguins = pl.read_csv("../data/penguins.csv", null_values="NA")
print(penguins)

shape: (344, 9)
┌───────┬───────────┬───────────┬────────────────┬───┬───────────────┬─────────────┬────────┬──────┐
│ rowid ┆ species   ┆ island    ┆ bill_length_mm ┆ … ┆ flipper_lengt ┆ body_mass_g ┆ sex    ┆ year │
│ ---   ┆ ---       ┆ ---       ┆ ---            ┆   ┆ h_mm          ┆ ---         ┆ ---    ┆ ---  │
│ i64   ┆ str       ┆ str       ┆ f64            ┆   ┆ ---           ┆ i64         ┆ str    ┆ i64  │
│       ┆           ┆           ┆                ┆   ┆ i64           ┆             ┆        ┆      │
╞═══════╪═══════════╪═══════════╪════════════════╪═══╪═══════════════╪═════════════╪════════╪══════╡
│ 1     ┆ Adelie    ┆ Torgersen ┆ 39.1           ┆ … ┆ 181           ┆ 3750        ┆ male   ┆ 2007 │
│ 2     ┆ Adelie    ┆ Torgersen ┆ 39.5           ┆ … ┆ 186           ┆ 3800        ┆ female ┆ 2007 │
│ 3     ┆ Adelie    ┆ Torgersen ┆ 40.3           ┆ … ┆ 195           ┆ 3250        ┆ female ┆ 2007 │
│ 4     ┆ Adelie    ┆ Torgersen ┆ null           ┆ … ┆ null          ┆ null

In [9]:
print(penguins.null_count().transpose(
    include_header=True, column_names=["null_count"]
))


shape: (9, 2)
┌───────────────────┬────────────┐
│ column            ┆ null_count │
│ ---               ┆ ---        │
│ str               ┆ u32        │
╞═══════════════════╪════════════╡
│ rowid             ┆ 0          │
│ species           ┆ 0          │
│ island            ┆ 0          │
│ bill_length_mm    ┆ 2          │
│ bill_depth_mm     ┆ 2          │
│ flipper_length_mm ┆ 2          │
│ body_mass_g       ┆ 2          │
│ sex               ┆ 11         │
│ year              ┆ 0          │
└───────────────────┴────────────┘


In [15]:
import chardet
def detect_encoding(filename: str) -> str:
    with open(filename, "rb") as f:
        raw_data = f.read()
        result = chardet.detect(raw_data)
        return result["encoding"]

print(detect_encoding("../data/directors.csv"))

EUC-JP


In [16]:
pl.read_csv("../data/directors.csv", encoding="EUC-JP")

name,born,country
str,i64,str
"""深作""",1930,"""日本"""
"""Verhoeven""",1938,"""オランダ"""
"""宮崎""",1942,"""日本"""
"""Tarantino""",1963,"""米国"""


In [19]:
songs = pl.read_excel("../data/top2000-2023.xlsx")
print(songs)

shape: (2_000, 4)
┌─────────┬─────────────────────────┬────────────────┬──────┐
│ positie ┆ titel                   ┆ artiest        ┆ jaar │
│ ---     ┆ ---                     ┆ ---            ┆ ---  │
│ i64     ┆ str                     ┆ str            ┆ i64  │
╞═════════╪═════════════════════════╪════════════════╪══════╡
│ 1       ┆ Bohemian Rhapsody       ┆ Queen          ┆ 1975 │
│ 2       ┆ Roller Coaster          ┆ Danny Vera     ┆ 2019 │
│ 3       ┆ Hotel California        ┆ Eagles         ┆ 1977 │
│ 4       ┆ Piano Man               ┆ Billy Joel     ┆ 1974 │
│ 5       ┆ Fix You                 ┆ Coldplay       ┆ 2005 │
│ …       ┆ …                       ┆ …              ┆ …    │
│ 1996    ┆ Charlie Brown           ┆ Coldplay       ┆ 2011 │
│ 1997    ┆ Beast Of Burden         ┆ Bette Midler   ┆ 1984 │
│ 1998    ┆ It Was A Very Good Year ┆ Frank Sinatra  ┆ 1968 │
│ 1999    ┆ Hou Van Mij             ┆ 3JS            ┆ 2008 │
│ 2000    ┆ Drivers License         ┆ Olivia Rodrigo

In [21]:
print(pl.read_csv("../data/stock/nvda/201?.csv"))

shape: (2_516, 8)
┌────────┬────────────┬───────────┬───────────┬───────────┬───────────┬───────────┬──────────┐
│ symbol ┆ date       ┆ open      ┆ high      ┆ low       ┆ close     ┆ adj close ┆ volume   │
│ ---    ┆ ---        ┆ ---       ┆ ---       ┆ ---       ┆ ---       ┆ ---       ┆ ---      │
│ str    ┆ str        ┆ f64       ┆ f64       ┆ f64       ┆ f64       ┆ f64       ┆ i64      │
╞════════╪════════════╪═══════════╪═══════════╪═══════════╪═══════════╪═══════════╪══════════╡
│ NVDA   ┆ 2010-01-04 ┆ 4.6275    ┆ 4.655     ┆ 4.5275    ┆ 4.6225    ┆ 4.240429  ┆ 80020400 │
│ NVDA   ┆ 2010-01-05 ┆ 4.605     ┆ 4.74      ┆ 4.605     ┆ 4.69      ┆ 4.30235   ┆ 72864800 │
│ NVDA   ┆ 2010-01-06 ┆ 4.6875    ┆ 4.73      ┆ 4.6425    ┆ 4.72      ┆ 4.32987   ┆ 64916800 │
│ NVDA   ┆ 2010-01-07 ┆ 4.695     ┆ 4.715     ┆ 4.5925    ┆ 4.6275    ┆ 4.245015  ┆ 54779200 │
│ NVDA   ┆ 2010-01-08 ┆ 4.59      ┆ 4.67      ┆ 4.5625    ┆ 4.6375    ┆ 4.254189  ┆ 47816800 │
│ …      ┆ …          ┆ …       

In [22]:
all_stocks = pl.read_csv("../data/stock/**/*.csv")
print(all_stocks)

shape: (18_476, 8)
┌────────┬────────────┬────────────┬────────────┬────────────┬────────────┬────────────┬──────────┐
│ symbol ┆ date       ┆ open       ┆ high       ┆ low        ┆ close      ┆ adj close  ┆ volume   │
│ ---    ┆ ---        ┆ ---        ┆ ---        ┆ ---        ┆ ---        ┆ ---        ┆ ---      │
│ str    ┆ str        ┆ f64        ┆ f64        ┆ f64        ┆ f64        ┆ f64        ┆ i64      │
╞════════╪════════════╪════════════╪════════════╪════════════╪════════════╪════════════╪══════════╡
│ ASML   ┆ 1999-01-04 ┆ 11.765625  ┆ 12.28125   ┆ 11.765625  ┆ 12.140625  ┆ 7.522523   ┆ 1801867  │
│ ASML   ┆ 1999-01-05 ┆ 11.859375  ┆ 14.25      ┆ 11.71875   ┆ 13.96875   ┆ 8.655257   ┆ 8241600  │
│ ASML   ┆ 1999-01-06 ┆ 14.25      ┆ 17.601563  ┆ 14.203125  ┆ 16.875     ┆ 10.456018  ┆ 16400267 │
│ ASML   ┆ 1999-01-07 ┆ 14.742188  ┆ 17.8125    ┆ 14.53125   ┆ 16.851563  ┆ 10.441495  ┆ 17722133 │
│ ASML   ┆ 1999-01-08 ┆ 16.078125  ┆ 16.289063  ┆ 15.023438  ┆ 15.796875  ┆ 9.787

In [24]:
import calendar

filenames = [
    f"../data/stock/asml/{year}.csv"
    for year in range(1999, 2024)
    if calendar.isleap(year)
]

filenames

['../data/stock/asml/2000.csv',
 '../data/stock/asml/2004.csv',
 '../data/stock/asml/2008.csv',
 '../data/stock/asml/2012.csv',
 '../data/stock/asml/2016.csv',
 '../data/stock/asml/2020.csv']